In [85]:
from dotenv import load_dotenv

In [128]:
MODEL = "llama2"
from langchain_community.llms import Ollama

In [129]:
model = Ollama(model=MODEL)

In [130]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [131]:
chain = model | parser
chain.invoke("how does it look now?")

'\nI\'m just an AI, I don\'t have a physical body or appearance, so I don\'t have a "look" in the classical sense. However, I can understand and respond to your messages in a text-based format. Is there something specific you would like to know or discuss?'

In [132]:

from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader
loader = DirectoryLoader('./', glob="./*.pdf", loader_cls=PyPDFLoader, show_progress=True)
documents = loader.load()

Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
100%|██████████| 1/1 [00:00<00:00,  8.60it/s]


In [133]:
documents = loader.load_and_split()
documents

Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
100%|██████████| 1/1 [00:00<00:00, 13.85it/s]


[Document(page_content='WelcomeAirBed&BreakfastBook rooms with locals, rather than hotels.1\nThis is a PowerPoint reproduction of an early AirBnB pitch deckvia Business Insider @http://www.businessinsider.com/airbnb-a-13-billion-dollar-startups-first-ever-pitch-deck-2011-9', metadata={'source': 'pitch.pdf', 'page': 0}),
 Document(page_content='Problem2Priceis an important concern for customers booking travel online.Hotelsleave you disconnected from the city and its culture. No easy way existsto book a room with a local or become a host.\nTemplate by PitchDeckCoach.com', metadata={'source': 'pitch.pdf', 'page': 1}),
 Document(page_content='Solution3A web platformwhere users can rent out their space to host travelers to:\nSAVE MONEYwhen traveling\nMAKEMONEYwhen hosting\nSHARECULTURElocal connection to the city\nTemplate by PitchDeckCoach.com', metadata={'source': 'pitch.pdf', 'page': 2}),
 Document(page_content='Market Validation4630,000on temporary housing site couchsurfing.com17,000tem

In [145]:
from langchain.prompts import PromptTemplate
template = """
answer the question based on the cotext below, if you cannot answer, simply reply i dont know
context: {context}
question: {question}
"""
prompt = PromptTemplate.from_template(template=template)
print(prompt.format(context="here is some context", question="here is a question"))


answer the question based on the cotext below, if you cannot answer, simply reply i dont know
context: here is some context
question: here is a question



In [146]:
chain = prompt | model | parser

In [147]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

In [148]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
embeddings = OllamaEmbeddings()

In [149]:
print(docs[0])

page_content='WelcomeAirBed&BreakfastBook rooms with locals, rather than hotels.1\nThis is a PowerPoint reproduction of an early AirBnB pitch deckvia Business Insider @http://www.businessinsider.com/airbnb-a-13-billion-dollar-startups-first-ever-pitch-deck-2011-9' metadata={'source': 'pitch.pdf', 'page': 0}


In [140]:
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

pc = Pinecone(api_key='')
index_name = "test-index-dim"

pc.create_index(
    name=index_name,
    dimension=4096,
    metric="cosine",
    spec=ServerlessSpec(
        cloud='aws', 
        region='us-east-1'
    ) 
) 



In [150]:
import os
from dotenv import load_dotenv
load_dotenv()
pinecone_api_key = os.environ.get('PINECONE_API_KEY')
index_name = "test-index-dim"
vectrostore = PineconeVectorStore.from_documents(docs, embedding=embeddings, index_name=index_name)

In [151]:
retriever = vectrostore.as_retriever()

In [152]:
retriever.invoke("")

[Document(page_content='Template by PitchDeckCoach.com1stTO MARKETHOST INCENTIVELorem ipsum dolor sit amet, consectetur adipiscing elit. Integer nec odio. Praesent libero. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Integer nec odio. Praesent libero. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Integer nec odio. Praesent libero. LIST ONCEEASE OF USEPROFILESLorem ipsum dolor sit amet, consectetur adipiscing elit. Integer nec odio. Praesent libero. Lorem ipsum dolor sit amet, consectetur', metadata={'page': 9.0, 'source': 'pitch.pdf'}),
 Document(page_content='Template by PitchDeckCoach.com1stTO MARKETHOST INCENTIVELorem ipsum dolor sit amet, consectetur adipiscing elit. Integer nec odio. Praesent libero. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Integer nec odio. Praesent libero. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Integer nec odio. Praesent libero. LIST ONCEEASE OF USEPROFILESLorem ipsum dolor sit amet, consectetur adipiscing

In [154]:
from operator import itemgetter
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser

)

chain.invoke({"question": ""})

"Based on the provided documents, I can identify some potential competitive advantages of AirBnB:\n\n1. Personalized experiences: By staying with locals, guests can have a more personalized and authentic experience, which is in line with Airbnb's mission to provide unique accommodations that allow travelers to feel at home.\n2. Local knowledge: Hosts are often knowledgeable about their area and can provide valuable insights and recommendations to guests, enhancing their overall travel experience.\n3. Cost-effective: Booking a room with a local through Airbnb can be more cost-effective than staying in a hotel, especially for longer stays or for travelers on a budget.\n4. Flexibility: Airbnb offers a wide range of accommodations, from shared rooms to entire homes and apartments, allowing guests to choose the level of privacy and comfort that best suits their needs.\n5. Community building: Airbnb fosters a sense of community among hosts and guests by providing a platform for people to con